## Automatic Music Generation using Python
### Analytics Vidhya - https://www.analyticsvidhya.com/blog/2020/01/how-to-perform-automatic-music-generation/

In [1]:
from numpy.random import seed
seed(1)
from tensorflow import set_random_seed
set_random_seed(2)

In [2]:
import tensorflow as tf
import re
import keras  

#dealing with midi files
from music21 import * 

#array processing
import numpy as np     
import os

#random number generator
import random         

#keras for building deep learning model
from keras.layers import * 
from keras.models import *
import keras.backend as K

Using TensorFlow backend.


In [3]:
os.listdir('schubert')

['schubert_D850_1.mid',
 'schubert_D850_2.mid',
 'schubert_D850_3.mid',
 'schubert_D850_4.mid',
 'schubert_D935_1.mid',
 'schubert_D935_2.mid',
 'schubert_D935_3.mid',
 'schubert_D935_4.mid',
 'schub_d760_1.mid',
 'schub_d760_2.mid',
 'schub_d760_3.mid',
 'schub_d760_4.mid',
 'schub_d960_1.mid',
 'schub_d960_2.mid',
 'schub_d960_3.mid',
 'schub_d960_4.mid',
 'schuim-1.mid',
 'schuim-2.mid',
 'schuim-3.mid',
 'schuim-4.mid',
 'schumm-1.mid',
 'schumm-2.mid',
 'schumm-3.mid',
 'schumm-4.mid',
 'schumm-5.mid',
 'schumm-6.mid',
 'schu_143_1.mid',
 'schu_143_2.mid',
 'schu_143_3.mid']

In [ ]:
#get a stream 
#parsed_midi = converter.parse(midi_test)
#play
#parsed_midi.show('midi')

In [ ]:
#plot the distribution of the notes played
#parsed_midi.plot('histogram','pitch')

In [ ]:
#group the midi in partitions by instrument. retrieve an iterator
#midi_parts = instrument.partitionByInstrument(parsed_midi) 
#for i in range(0,len(midi_parts)):
#    partition = str(midi_parts[i])
#    instrument = re.sub('[<>]', '', partition).split()[1]
#    print(i+1, instrument)

In [ ]:
#midi_parts[1].elements[6].pitch

In [12]:
# Define a function for reading the MIDI files. 
# It returns the set of notes and chords present in the musical file.
def read_midi(file):
    notes=[]
    notes_to_parse = None
    #parsing a midi file
    midi = converter.parse(file)    
    #grouping based on different instruments    
    s2 = instrument.partitionByInstrument(midi)
    
    #Looping over all the instruments
    for part in s2.parts:
    #select elements of only piano
        if 'Piano' in str(part): 
            notes_to_parse = part.recurse() 
            #finding whether a particular element is note or a chord
            for element in notes_to_parse:
                if isinstance(element, note.Note):
                    notes.append(str(element.pitch))
                elif isinstance(element, chord.Chord):
                    notes.append('.'.join(str(n) for n in element.normalOrder))        
    return notes

In [26]:
#test file
os.listdir('schubert/')
test = 'schubert/'+os.listdir('schubert/')[0]

In [35]:
#read all the filenames in dir
files = ['schubert/'+i for i in os.listdir('schubert') if i.endswith(".mid")]
print(files)

['schubert/schubert_D850_1.mid', 'schubert/schubert_D850_2.mid', 'schubert/schubert_D850_3.mid', 'schubert/schubert_D850_4.mid', 'schubert/schubert_D935_1.mid', 'schubert/schubert_D935_2.mid', 'schubert/schubert_D935_3.mid', 'schubert/schubert_D935_4.mid', 'schubert/schub_d760_1.mid', 'schubert/schub_d760_2.mid', 'schubert/schub_d760_3.mid', 'schubert/schub_d760_4.mid', 'schubert/schub_d960_1.mid', 'schubert/schub_d960_2.mid', 'schubert/schub_d960_3.mid', 'schubert/schub_d960_4.mid', 'schubert/schuim-1.mid', 'schubert/schuim-2.mid', 'schubert/schuim-3.mid', 'schubert/schuim-4.mid', 'schubert/schumm-1.mid', 'schubert/schumm-2.mid', 'schubert/schumm-3.mid', 'schubert/schumm-4.mid', 'schubert/schumm-5.mid', 'schubert/schumm-6.mid', 'schubert/schu_143_1.mid', 'schubert/schu_143_2.mid', 'schubert/schu_143_3.mid']


In [36]:
#reading each midi file
all_notes=[]
for i in files:
    all_notes.append(read_midi(i))

#notes and chords of all the midi files
notes = [element for notes in all_notes for element in notes]

In [37]:
#length of a input sequence
no_of_timesteps = 128      

#no. of unique notes
n_vocab = len(set(notes))  

#all the unique notes
pitch = sorted(set(item for item in notes))  

#assign unique value to every note
note_to_int = dict((note, number) for number, note in enumerate(pitch))  

#preparing input and output sequences
X = []
y = []
for notes in all_notes:
    for i in range(0, len(notes) - no_of_timesteps, 1):
        input_ = notes[i:i + no_of_timesteps]
        output = notes[i + no_of_timesteps]
        X.append([note_to_int[note] for note in input_])
        y.append(note_to_int[output])

The input to the Convolution 1D or LSTM has to be in the form of (samples, timesteps, features). So, let us reshape out input array according to the required shape. Notice that the number of features in each timestep is 1

In [38]:
#reshaping input X into an array of 'n' samples of size 128 each, for a total length equal to the intial length
X = np.reshape(X, (len(X), no_of_timesteps, 1))
X

array([[[ 95],
        [ 79],
        [229],
        ...,
        [301],
        [255],
        [288]],

       [[ 79],
        [229],
        [ 92],
        ...,
        [255],
        [288],
        [255]],

       [[229],
        [ 92],
        [229],
        ...,
        [288],
        [255],
        [275]],

       ...,

       [[ 92],
        [157],
        [289],
        ...,
        [218],
        [218],
        [218]],

       [[157],
        [289],
        [234],
        ...,
        [218],
        [218],
        [218]],

       [[289],
        [234],
        [258],
        ...,
        [218],
        [218],
        [218]]])

In [39]:
#normalizing the inputs
X = X / float(n_vocab) 

In [40]:
X

array([[[0.3125    ],
        [0.25986842],
        [0.75328947],
        ...,
        [0.99013158],
        [0.83881579],
        [0.94736842]],

       [[0.25986842],
        [0.75328947],
        [0.30263158],
        ...,
        [0.83881579],
        [0.94736842],
        [0.83881579]],

       [[0.75328947],
        [0.30263158],
        [0.75328947],
        ...,
        [0.94736842],
        [0.83881579],
        [0.90460526]],

       ...,

       [[0.30263158],
        [0.51644737],
        [0.95065789],
        ...,
        [0.71710526],
        [0.71710526],
        [0.71710526]],

       [[0.51644737],
        [0.95065789],
        [0.76973684],
        ...,
        [0.71710526],
        [0.71710526],
        [0.71710526]],

       [[0.95065789],
        [0.76973684],
        [0.84868421],
        ...,
        [0.71710526],
        [0.71710526],
        [0.71710526]]])

In [41]:
# LSTM
def lstm():
    model = Sequential()
    model.add(LSTM(128,return_sequences=True))
    model.add(LSTM(128))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    return model

In [42]:
# WaveNet
K.clear_session()
def simple_wavenet():
    no_of_kernels=64
    num_of_blocks= int(np.sqrt(no_of_timesteps)) - 1   #no. of stacked conv1d layers
    model = Sequential()
    for i in range(num_of_blocks):
        model.add(Conv1D(no_of_kernels,3,dilation_rate=(2**i),padding='causal',activation='relu'))
        model.add(Conv1D(1, 1, activation='relu', padding='causal'))
        model.add(Flatten())
        model.add(Dense(128, activation='relu'))
        model.add(Dense(n_vocab, activation='softmax'))
        model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
        return model

In [43]:
# Define the callback to save the model after every 50 epochs:
mc = keras.callbacks.ModelCheckpoint('model{epoch:03d}.h5', save_weights_only=False, period=50)

In [44]:
# Instantiate and train the model with a batch size of 128:
model = simple_wavenet()
model.fit(X,np.array(y), epochs=300, batch_size=128,callbacks=[mc])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Epoch 1/300
63757/63757 [==============================] - 5s 77us/step - loss: 4.8271
Epoch 2/300
63757/63757 [==============================] - 4s 70us/step - loss: 4.6417
Epoch 3/300
63757/63757 [==============================] - 4s 70us/step - loss: 4.5759
Epoch 4/300
63757/63757 [==============================] - 5s 72us/step - loss: 4.5241
Epoch 5/300
63757/63757 [==============================] - 5s 71us/step - loss: 4.4616
Epoch 6/300
63757/63757 [==============================] - 5s 72us/step - loss: 4.3947
Epoch 7/300
63757/63757 [==============================] - 5s 78us/step - loss: 4.3332
Epoch 8/300
63757/63757 [==============================] - 5s 77us/step - loss: 4.2755
Epoch 9/300
63757/63757 [==============================] - 5s 85us/step - loss: 4.2214
Epoch 10/300
63757/63757 [==============================] - 5s 79us/step - loss: 4.1765
Epoch 11/300
63757/63757 [=====================

63757/63757 [==============================] - 5s 75us/step - loss: 3.5569
Epoch 88/300
63757/63757 [==============================] - 5s 73us/step - loss: 3.5572
Epoch 89/300
63757/63757 [==============================] - 4s 70us/step - loss: 3.5522
Epoch 90/300
63757/63757 [==============================] - 4s 70us/step - loss: 3.5519
Epoch 91/300
63757/63757 [==============================] - 5s 76us/step - loss: 3.5498
Epoch 92/300
63757/63757 [==============================] - 5s 74us/step - loss: 3.5469
Epoch 93/300
63757/63757 [==============================] - 5s 72us/step - loss: 3.5466
Epoch 94/300
63757/63757 [==============================] - 4s 70us/step - loss: 3.5437
Epoch 95/300
63757/63757 [==============================] - 5s 71us/step - loss: 3.5418
Epoch 96/300
63757/63757 [==============================] - 5s 71us/step - loss: 3.5381
Epoch 97/300
63757/63757 [==============================] - 5s 71us/step - loss: 3.5386
Epoch 98/300
63757/63757 [===================

63757/63757 [==============================] - 6s 87us/step - loss: 3.4425
Epoch 180/300
63757/63757 [==============================] - 6s 95us/step - loss: 3.4416
Epoch 181/300
63757/63757 [==============================] - 6s 91us/step - loss: 3.4430
Epoch 182/300
63757/63757 [==============================] - 6s 92us/step - loss: 3.4392
Epoch 183/300
63757/63757 [==============================] - 6s 96us/step - loss: 3.4384
Epoch 184/300
63757/63757 [==============================] - 6s 88us/step - loss: 3.4386
Epoch 185/300
63757/63757 [==============================] - 5s 79us/step - loss: 3.4375
Epoch 186/300
63757/63757 [==============================] - 5s 76us/step - loss: 3.4386
Epoch 187/300
63757/63757 [==============================] - 5s 81us/step - loss: 3.4365
Epoch 188/300
63757/63757 [==============================] - 6s 87us/step - loss: 3.4361
Epoch 189/300
63757/63757 [==============================] - 6s 88us/step - loss: 3.4350
Epoch 190/300
63757/63757 [========

63757/63757 [==============================] - 4s 69us/step - loss: 3.3894
Epoch 271/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3875
Epoch 272/300
63757/63757 [==============================] - 4s 70us/step - loss: 3.3893
Epoch 273/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3884
Epoch 274/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3869
Epoch 275/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3871
Epoch 276/300
63757/63757 [==============================] - 5s 71us/step - loss: 3.3851
Epoch 277/300
63757/63757 [==============================] - 4s 68us/step - loss: 3.3856
Epoch 278/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3851
Epoch 279/300
63757/63757 [==============================] - 4s 69us/step - loss: 3.3826
Epoch 280/300
63757/63757 [==============================] - 5s 72us/step - loss: 3.3827
Epoch 281/300
63757/63757 [========

Inference phase
In the inference phase, we will try to generate new samples. Let’s see how to do that:

- Select a random array of sample values as a starting point to model
- Now, the model outputs the probability distribution over all the samples
- Choose the value with the maximum probability and append it to an array of samples
- Delete the first element and pass as an input for the next iteration
- Repeat steps 2 and 4 for a certain number of iterations
 

In [52]:
def generate_music(model, pitch, no_of_timesteps, pattern):
    
    int_to_note = dict((number, note) for number, note in enumerate(pitch))
    prediction_output = []
    
    # generate 50 elements
    for note_index in range(50):
        #reshaping array to feed into model
        input_ = np.reshape(pattern, (1, len(pattern), 1))
        #predict the probability and choose the maximum value
        proba = model.predict(input_, verbose=0)
        index = np.argmax(proba)
        #convert integer back to the element
        pred = int_to_note[index]
        prediction_output.append(pred)
        pattern = list(pattern)
        pattern.append(index/float(n_vocab))
        #leave the first value at index 0
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [53]:
def convert_to_midi(prediction_output):
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # Specify duration between 2 notes
        offset += 0.5
       # offset += random.uniform(0.5,0.9)

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='music.mid')

In [54]:
#Select random chunk for the first iteration
start = np.random.randint(0, len(X)-1)
pattern = X[start]
#load the best model
model=load_model('model300.h5')
#generate and save music
music = generate_music(model,pitch,no_of_timesteps,pattern)
convert_to_midi(music)

In [55]:
converter.parse([i for i in os.listdir() if 'music' in str(i)][0]).show('midi')